In [1]:
import datetime
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pystac
import rasterio.features
import rioxarray as rioxr
import shapely.affinity
import shapely.geometry
import xarray as xr

from pystac import Catalog, Item, Asset

# Generate test catalog

In [2]:
catalog_id = "test-catalog"

In [3]:
# make sub dir for the assets
path = pathlib.Path(f"./{catalog_id}_assets")
path.mkdir(exist_ok=True)

In [4]:
# generate the assets
dim_size = 100

asset_paths = []
for offset_x in (0., dim_size):
    for offset_y in (0., dim_size):
        da = xr.DataArray(
            np.random.random((dim_size, dim_size)),
            coords={
                'x': np.arange(dim_size) + offset_x,
                'y': np.arange(dim_size) + offset_y
            },
            dims=['y', 'x']
        )
        da = da.rio.set_crs("EPSG:28992")
        da = da.rio.set_nodata(np.nan)
        polygon = shapely.geometry.Polygon.from_bounds(
            offset_x,
            offset_y,
            offset_x + dim_size,
            offset_y + dim_size
            )
        polygon = shapely.affinity.rotate(polygon, 45)
        da = da.rio.clip([polygon])
    
        asset_num = len(asset_paths) + 1
        asset_path = path / f"tile_{asset_num}.tif"
        da.rio.to_raster(asset_path.as_posix())
        asset_paths.append(asset_path)


In [5]:
# create the catalog
catalog = Catalog(
    id=catalog_id,
    description="This is a test catalog"
)

In [6]:
# add items
for asset_path in asset_paths:
    da = rioxr.open_rasterio(asset_path.as_posix()).squeeze()
    
    asset = Asset(
        href=asset_path.as_posix(),  # link to asset
        title=asset_path.name,
        media_type=pystac.MediaType.GEOTIFF
    )
    
    # get tile bounds in WGS84, which is the standard in the GeoJSON format
    bbox = da.rio.transform_bounds("WGS84")
    
    # determine footprint geometry
    mask = da.isnull()

    polygons_and_values = rasterio.features.shapes(
        mask.astype("int16").values,  # satisfy rasterio type requirements 
        transform=mask.rio.transform()
    )
    polygons = (shapely.geometry.shape(pol) for pol, val in polygons_and_values if val == 0.)
    polygons = geopandas.GeoSeries(
        polygons, 
        crs=mask.spatial_ref.crs_wkt
    )
    polygons = polygons.to_crs("WGS84")

    geometry = shapely.geometry.mapping(polygons[0])
    
    # create Item object
    item = Item(
        id=asset_path.stem,
        geometry=geometry,
        bbox=bbox,
        datetime=datetime.datetime.utcnow(),
        properties={}
    )

    item.add_asset(asset_path.stem, asset)
    catalog.add_item(item)

In [7]:
catalog.describe()

* <Catalog id=test-catalog>
  * <Item id=tile_1>
  * <Item id=tile_2>
  * <Item id=tile_3>
  * <Item id=tile_4>


In [8]:
catalog.normalize_and_save(
    f"./{catalog_id}",
    'SELF_CONTAINED'
)